# **Recomendacion basada en popularidad**
---

In [1]:
import numpy as np
import pandas as pd

#Recomendacion por popularidad

df=pd.read_csv('../../Dataset_Apart/Cleaned/DatasetAirbnb_Cleaned_v1.csv')
# Definir las columnas de calificaciones
rating_columns = ['Limpieza', 'Veracidad', 'Llegada', 'Comunicacion', 'Ubicacion', 'Calidad']

# Llenar valores faltantes en las columnas de calificaciones con la media
df[rating_columns] = df[rating_columns].fillna(df[rating_columns].mean())

# Calcular el promedio de las calificaciones para cada apartamento
df['Promedio_Ratings'] = df[rating_columns].mean(axis=1)

# Definir la puntuación de popularidad como una combinación de evaluaciones y promedio de ratings
# Ajusta los pesos según la importancia que quieras dar a cada factor
peso_evaluaciones = 0.6
peso_ratings = 0.4

df['Puntuacion_Popularidad'] = peso_evaluaciones * df['Evaluaciones'] + peso_ratings * df['Promedio_Ratings']

def recomendador_por_popularidad(df, top_n=5):
    return df.sort_values(by='Puntuacion_Popularidad', ascending=False).head(top_n)

# Obtener los 3 apartamentos más populares según la nueva métrica
recomendaciones_populares = recomendador_por_popularidad(df, top_n=3)
print(recomendaciones_populares[['ID', 'Titulo', 'Evaluaciones', 'Promedio_Ratings', 'Puntuacion_Popularidad']])


                      ID                                  Titulo  \
79   6119377963866422626      Estudio Biarritz, estudio, 2 pers.   
185  3262728650081535420               Estrella ii 2d Spaniahome   
120  2130813439338571442  Nido acogedor entre la bahía y el mar.   

     Evaluaciones  Promedio_Ratings  Puntuacion_Popularidad  
79         6854.0          4.794622             4114.317849  
185        2076.0          4.794622             1247.517849  
120        1394.0          4.750000              838.300000  


# **Recomendacion por similaridad**
---

La idea esque el usuario haga una preseleccion de aquellos que le gustan y recomendarle los mas similares a estos. A continuacion se hace una prueba simplificada midiendo la similaridad por coseno.

In [4]:
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import StandardScaler

# Seleccionar características para calcular la similitud
features = ['ID','Titulo','Descripcion Simple','URL','url_img']

# Escalar características
scaler = StandardScaler()
scaled_features = scaler.fit_transform(df[features])

# Calcular la similitud del coseno
similarity_matrix = cosine_similarity(scaled_features)

# Función para recomendar apartamentos similares
def recomendador_por_similitud(df, apartment_id, top_n=5):
    # Encontrar el índice del apartamento
    idx = df.index[df['ID'] == apartment_id][0]
    
    # Obtener la similitud de los apartamentos
    similarity_scores = list(enumerate(similarity_matrix[idx]))
    
    # Ordenar los apartamentos por similitud
    similarity_scores = sorted(similarity_scores, key=lambda x: x[1], reverse=True)
    
    # Obtener los índices de los apartamentos más similares
    similar_apartments_indices = [i[0] for i in similarity_scores[1:top_n+1]]
    
    return df.iloc[similar_apartments_indices]

# Obtener apartamentos similares a un apartamento específico
apartamento_id = 1200688382914319681  # ID del apartamento de ejemplo
recomendaciones_similares = recomendador_por_similitud(df, apartamento_id, top_n=4)
recomendaciones_similares


,ID,Titulo,Descripcion Simple,Evaluaciones,Tipo,Precio,URL,Limpieza,Veracidad,Llegada,...,Camas,Baños,Dormitorios,Baño Compartido,Wifi,Mascotas,Piscina,Parking,Promedio_Ratings,Puntuacion_Popularidad
67,-7545000407210804405,"Frente al mar con piscina, 3dorm. Parking, Fibra",Alojamiento entero: apto. residencial en Daimu...,53.0,A pie de playa,87.0,https://www.airbnb.es/rooms/31642583?adults=1&...,4.9,4.9,4.9,...,5.0,2.000000,3.0,0,1.0,1.0,1.0,1.0,4.900000,33.760000
421,6895992565345467060,Magnifico Chalet en la Sierra de Madrid,"Habitación en Valdencina, España",90.0,En el campo,172.0,https://www.airbnb.es/rooms/14770870?adults=1&...,4.9,4.9,5.0,...,5.0,2.033012,3.0,0,1.0,1.0,1.0,1.0,4.916667,55.966667
769,-2912441540787268215,Casa de Madera II,"Alojamiento entero: cabaña en Bicorp, España",70.0,Cabañas,37.0,https://www.airbnb.es/rooms/31512931?adults=1&...,5.0,4.9,4.9,...,6.0,1.000000,3.0,0,1.0,1.0,1.0,1.0,4.933333,43.973333
596,-7843512998668970045,La cabaña de Flo,Alojamiento entero: chalet en Saint-Christophe...,115.0,Cabañas,150.0,https://www.airbnb.es/rooms/47299709?adults=1&...,4.9,5.0,5.0,...,5.0,2.000000,3.0,0,1.0,1.0,1.0,1.0,4.933333,70.973333
